<a href="https://colab.research.google.com/github/marty916/AI-Training-Colab-Notebooks/blob/main/LangGraph_QuickStart_OpenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 1: Build a Basic Chatbot using LangGraph

### Step 1: Application Setup

In [ ]:
# Install required libraries
!pip install -U langgraph langchain_openai

In [ ]:
# Retrieve api_key from colab secrets
from google.colab import userdata
api_key=userdata.get('OPENAI_API_KEY')

### Step 2: Create the LangGraph state graph

In [ ]:
# Annotated standard in Python 3.8
# TypedDict standard in Python 3.9
# Using an ealier version of python use `typing_extension` instead of `typing`
from typing import Annotated, TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

### Step 3: Add the `chatbot` node. This node calls the LLM.


In [ ]:
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(api_key=api_key, model="gpt-4")


def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}


# The first argument is the unique node name
# The second argument is the function or object that will be called whenever
# the node is used.
graph_builder.add_node("chatbot", chatbot)

### Step 4: Add an entry point edge,  telling our graph where to start each time we run it.

In [ ]:
graph_builder.add_edge(START, "chatbot")

### Step 5: Add a finish point edge, instructing our graph **any time this node is run, you can exit.**

In [ ]:
graph_builder.add_edge("chatbot", END)

### Step 6: Compile the graph

In [ ]:
graph = graph_builder.compile()

In [ ]:
# Let's visualize the graph. We don't need to do this but when we are building
# and testing it's nice to see it.
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

### Step 7: Run the chatbot!

Note: You can exit the chat loop any time by typeing `quit`, `exit`, or `q`

In [ ]:
while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Awesome you created your first LangGraph chatbot. Goodbye!")
        break
    for event in graph.stream({"messages": ("user", user_input)}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)